# A very simple way to monitor market risk with Atoti: Value at Risk 101

Financial institutions all have to find a balance between profit and risk. The more risk taken the higher the profit can be, but then if we want to avoid collapses such as that of Lehman Brothers in 2008, risk has to be controlled.  

There are several kinds of risk:
- Shortfall of a counterparty also know as credit risk: This is the risk that a borrower cannot pay its credit
- Market risk: This is the risk that certain assets could lose their value. For example one might invest in wine bottle in the hope that they gain value with age while they might not.

Market risk is widely monitored in finance. Institutions have large portfolios with a lot of assets, and forecasting the value of each asset is simply impossible as COVID-19 kindly reminded it to us recently. The key is then to assess what are the (statistical) chances that the value of certain assets remain in a certain enveloppe and what are potential losses. This is where the value at risk – or VaR – comes into action.

In this notebook we will show you how we can compute and aggregate pretty easily this non-linear indicator with Atoti, and then perform simulations around it.

## Importing the necessary libraries

In [1]:
import pandas as pd
import atoti

## Data Loading
#### Positions, Instruments, PnL vectors per instrument

In [2]:
session = atoti.create_session()

#### Import your data from any standard data source

In [3]:
instruments = session.read_csv('http://data.atoti.io/notebooks/var/instruments.csv', keys=['instrument_code'], store_name="Instruments")

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'http:\\data.atoti.io\\notebooks\\var\\instruments.csv'

In [ ]:
instruments.head()

In [ ]:
analytics = session.read_csv('http://data.atoti.io/notebooks/var/instruments_pricing_vol_depth_250.csv', keys=['instrument_code'], store_name="Instruments Analytics", sep="|", array_sep=";")

In [ ]:
analytics.head()

In [ ]:
positions_store_types = {
    "quantity": atoti.types.DOUBLE,
    "purchase_price": atoti.types.DOUBLE,
}

In [ ]:
positions = session.read_csv('http://data.atoti.io/notebooks/var/positions.csv', keys=['instrument_code','book_id'], store_name="Positions", types=positions_store_types)

In [ ]:
positions.head()

### Data model and cube

In [ ]:
positions.join(instruments)
instruments.join(analytics)

In [ ]:
cube = session.create_cube(positions, "Positions")

In [ ]:
m = cube.measures
h = cube.hierarchies
l = cube.levels
cube

In [ ]:
cube.visualize()

### Perform a query on the cube using python API
#### (PnL per Underlying)

In [ ]:
m["Quantity"] = atoti.agg.sum(m["quantity.SUM"])
m["PnL"] = atoti.agg.sum(m["Quantity"] * m["pnl.VALUE"], scope=atoti.scope.origin('instrument_code'))
# cube.query(m["PnL"], levels=[l['underlying']])

### Open the Visual Environment

In [ ]:
cube.visualize(name="PnL Pivot Table")

In [ ]:
cube.visualize("PnL by Type Chart")

### Introduction of collaboration tools

In [ ]:
session.url

### Custom Hierarchy
* #### Import a file with the definition of the trading book hierarchy
* #### Enrich the model on the fly, create a new multi-level hierarchy on the fly
* #### Use it in the visual environment

In [ ]:
trading_desks = session.read_csv('http://data.atoti.io/notebooks/var/trading_desk.csv', keys=['book_id'], store_name="Trading Desk")
positions.join(trading_desks)

h["Trading Book Hierarchy"] = {
    'Business Unit':l['business_unit'],
    'Sub Business Unit':l['sub_business_unit'],
    'Trading Desk' : l['trading_desk'], 
    'Book':l['book']
}

#### Modification of the cube's structure on the fly

In [ ]:
cube.visualize("Business Hierarchy Pivot Table")

### Value at Risk
* #### Scale instrument PnL vectors per position
* #### Aggregate scaled vectors
* #### Calculate percentile of the aggregated vector, based on confidence level
* #### Use it on the fly in the visual environment

In [ ]:
scaled_pnl_vector = m["Quantity"] * m["pnl_vector.VALUE"]
m['Position Vector'] = atoti.agg.sum(scaled_pnl_vector, scope=atoti.scope.origin('instrument_code'))

m["Confidence Level"] = 0.95
m["VaR"] = atoti.array.percentile(m["Position Vector"], m["Confidence Level"])

In [ ]:
cube.visualize()

* #### Create a Measure Simulation on the Confidence Level
* #### Define various Confidence Levels
* #### They appear as a new hierarchy in the visual environment

In [ ]:
confidence_levels = cube.setup_simulation("Confidence Level", replace=[m["Confidence Level"]]).scenarios;

confidence_levels["90%"] = 0.90
confidence_levels["98%"] = 0.98
confidence_levels["99%"] = 0.99
confidence_levels["Worst"] = 1.0

In [ ]:
cube.visualize("VAR 98% on SBU")

### Marginal VaR
* #### Calculate Parent's VaR excluding the child's contribution
* #### Calculate the difference with ther original Parent VaR

In [ ]:
m["Parent Position Vector Ex"] = atoti.agg.sum(
    m["Position Vector"],
    scope=atoti.scope.siblings(h["Trading Book Hierarchy"], exclude_self=True),
)

In [ ]:
m["Parent VaR Ex"] = atoti.array.percentile(m["Parent Position Vector Ex"], m["Confidence Level"])
m["Parent VaR"] = atoti.parent_value(m["VaR"], h["Trading Book Hierarchy"])
m["Marginal VaR"] = m["Parent VaR"] - m["Parent VaR Ex"]

In [ ]:
cube.visualize()

In [ ]:
cube.visualize()

In [ ]:
cube.visualize()

## Model Comparison
* #### Price instruments with another model,
* #### Load as another scenario,
* #### Compare using all existing hierarchies and KPIs

In [ ]:
new_analytics_file = 'data/instruments_pricing_vol_depth_150.csv'
analytics.scenarios['Model short Volatility'].load_csv(new_analytics_file, sep="|", array_sep=";")

In [ ]:
cube.visualize("PnL comparaison Model Short Volatility")

In [ ]:
cube.visualize("VAR Comaraison Model Short Volatility")